# Bật GPU
"Runtime" -> "Change runtime type", select "Hardware Accelerator" -> "GPU" and click "SAVE"

In [1]:
!nvidia-smi

Tue Aug 11 07:38:14 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    31W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Cài đặt các gói cần thiết

In [2]:
%%shell
pip3 install transformers
git clone https://github.com/pytorch/fairseq
cd fairseq
pip3 install ./

     |████████████████████████████████| 778kB 2.6MB/s 
     |████████████████████████████████| 890kB 6.7MB/s 
     |████████████████████████████████| 1.1MB 13.1MB/s 
     |████████████████████████████████| 3.0MB 19.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=a9fc550498b176fd40ab929a752a8f97d3cc7b0cf1fb03c3fd515da9b9195fcc
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
Cloning into 'fairseq'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 17147 (delta 26), reused 34 (delta 14), pack-reused 17086
Receiving objects: 100% (17147/17147), 7.93 MiB | 6.11 MiB/s, done.
Resolving deltas: 100% (12630/12630), done.
Processing /content/fairseq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend

In [3]:
# Check version
import torch
print(torch.__version__)

1.6.0+cu101


## Tokenizer

In [4]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Mount Google Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive/')
root_dir = "/content/drive/My\ Drive/MasterThesis/CoLab/code_final"
src_dir = "%s/src" % root_dir

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


## Translate

In [6]:
import sys
sys.path.insert(0, '/content/drive/My Drive/MasterThesis/CoLab/code_final')
import src

model = src.TransformerWithBertModel.from_pretrained(
    "/content/drive/My Drive/MasterThesis/CoLab/code_final/model_stage2",
    checkpoint_file='checkpoint_best.pt',
    data_name_or_path="/content/drive/My Drive/MasterThesis/CoLab/code_final/data",
    bert_model="bert-base-multilingual-cased",
    )
model.eval() # disable dropout

GeneratorHubInterface(
  (models): ModuleList(
    (0): TransformerWithBertModel(
      (encoder): TransformerWithBertEncoder(
        (bert_model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(119547, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0): BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=True)
                    (key): Linear(in_features=768, out_features=768, bias=True)
                    (value): Linear(in_features=768, out_features=768, bias=True)
                    (dropout): Dropout(p=0.1, i

In [ ]:
while True:
    print('Nhập câu sai chính tả (q để thoát):')
    line = input()
    if 'q' == line.rstrip():
        break
    
    tokens = " ".join(['[CLS]'] + tokenizer.tokenize(line.strip()) + ['[SEP]'])
    print("Tokenized: %s" % tokens)

    result = model.translate(tokens)
    print("Output: %s" % result)

    detokenized = ''.join(result).replace(' ', '').replace('▁', ' ')
    print("Detokenized: %s" % detokenized)

    print('---------------------------------------------------------')


Nhập câu sai chính tả (q để thoát):
Học xinh đi học trể.
Tokenized: [CLS] Học xin ##h đi học tr ##ể . [SEP]
Output: ▁Học ▁sinh ▁đi ▁học ▁trễ .
Detokenized:  Học sinh đi học trễ.
---------------------------------------------------------
Nhập câu sai chính tả (q để thoát):
Tôi iêu Diệt Nam.
Tokenized: [CLS] T ##ôi i ##êu Di ##ệt Nam . [SEP]
Output: ▁Tôi ▁điêu ▁Việt ▁Nam .
Detokenized:  Tôi điêu Việt Nam.
---------------------------------------------------------
Nhập câu sai chính tả (q để thoát):
Tôi yêu Diệt Nam.
Tokenized: [CLS] T ##ôi yêu Di ##ệt Nam . [SEP]
Output: ▁Tôi ▁yêu ▁Việt ▁Nam .
Detokenized:  Tôi yêu Việt Nam.
---------------------------------------------------------
Nhập câu sai chính tả (q để thoát):
Tôi iêu Việt Nam.
Tokenized: [CLS] T ##ôi i ##êu Việt Nam . [SEP]
Output: ▁Tôi ▁điêu ▁Việt ▁Nam .
Detokenized:  Tôi điêu Việt Nam.
---------------------------------------------------------
Nhập câu sai chính tả (q để thoát):
Tôi yêu iệt Nam.
Tokenized: [CLS] T ##ôi yêu i ##ệt 